
# 💼 Resume & Job Description ADA Scorer (Gen-AI Based)
This Jupyter Notebook calculates an **ADA Score** (Applicant–Job Description Alignment) using **Sentence-BERT** embeddings.

### ✅ Requirements
Install required libraries before running:
```bash
pip install pandas sentence-transformers scikit-learn
```


In [16]:

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')



In [11]:

# 🔽 Load your datasets (Make sure they are in the same folder as this notebook)
resume_df = pd.read_csv("RESUME_SCREENING_DATASET.csv")
job_df = pd.read_csv("JOB_POSTS_DATASET.csv")

# Preview
display(resume_df.head(2))
display(job_df.head(2))


,resume_id,name,email,phone,age,education,university,graduation_year,total_experience,job_history,skills,certifications,projects,last_job_title,last_company,summary,location,languages,is_relevant
0,9096da72-3f8a-4cef-a75f-26dae774ad5c,Alyssa Ramos,melissa75@example.net,+1-449-877-8232,31,High School,NaN,2019,8,"[{'title': 'Engineering Manager', 'duration': ...","Python, Java, JavaScript, C++, SQL, NoSQL, AWS...",Yes,1,Engineering Manager,Frost and Sons,My knowledge worker buy support fly during. Ma...,South Donna,"English, Chinese",0
1,1d3e582d-7ae0-474c-b99f-dc43cdae541d,Ann Navarro,christinemiranda@example.org,001-498-233-9127,48,Bachelor's Degree,Polytechnic Institute,2011,10,"[{'title': 'Data Analyst', 'duration': 10, 'co...","Python, Java, JavaScript, SQL, NoSQL, Azure, K...",No,1,Data Analyst,Cantu-Cox,Father head listen affect guy the then themsel...,Cooperbury,English,0


,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False


In [18]:

# 🤖 Load pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')


In [20]:

# 🧠 ADA Score calculator function
def get_ada_score(resume_text, jd_text):
    resume_emb = model.encode([resume_text], convert_to_tensor=True)
    jd_emb = model.encode([jd_text], convert_to_tensor=True)
    score = util.cos_sim(resume_emb, jd_emb).item()
    return round(score * 100, 2)


In [22]:

# 📝 Example: First resume vs first job post
resume_sample = resume_df.iloc[0]
jd_sample = job_df.iloc[0]

# Combine resume fields (summary + skills + last job title)
resume_text = f"{resume_sample['summary']} Skills: {resume_sample['skills']} Job Title: {resume_sample['last_job_title']}"
jd_text = jd_sample['jobpost']

score = get_ada_score(resume_text, jd_text)
print(f"✅ ADA Score between resume and JD: {score}%")


✅ ADA Score between resume and JD: 19.21%


In [24]:

# 🔄 Optional: Score all resumes against one job description
jd_text = job_df.iloc[0]['jobpost']

scores = []
for i, row in resume_df.iterrows():
    resume_text = f"{row['summary']} Skills: {row['skills']} Job Title: {row['last_job_title']}"
    score = get_ada_score(resume_text, jd_text)
    scores.append((row['name'], score))

# Top matches
sorted(scores, key=lambda x: x[1], reverse=True)[:5]


[('Robert Mcbride', 43.43),
 ('Molly Harris', 40.26),
 ('Bradley Wallace', 39.62),
 ('Johnny Castillo', 39.45),
 ('Bryce Perez', 39.36)]